# IT350 Project

### Libraries

In [ ]:
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd
import json
import os.path

headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}

In [ ]:
url = "https://chiqa.nlm.nih.gov/answer?question="

#### AIDS Dataset

##### Preprocessing

In [ ]:
df = pd.read_csv("aids.csv") 
df

,Unnamed: 0,Disease,Question,MedScape
0,0,aids,What is HIV and how is it transmitted?,Practice EssentialsHuman immunodeficiency viru...
1,1,aids,What are the signs and symptoms of HIV infection?,Practice EssentialsHuman immunodeficiency viru...
2,2,aids,What are risk factors for HIV exposure?,Practice EssentialsHuman immunodeficiency viru...
3,3,aids,What are the recommendations for HIV screening?,HIV screening recommendations include the foll...
4,4,aids,What is the role of CD4 T-cell count in the ri...,The CD4 T-cell count reliably reflects the cur...
...,...,...,...,...
209,209,aids,Which medications in the drug class Antiretrov...,Combination products are valuable to patient c...
210,210,aids,Which medications in the drug class CYP3A4 Inh...,"Boosting agents (eg, ritonavir, cobicistat) ma..."
211,211,aids,Which medications in the drug class Antibiotic...,Effective antiretroviral therapy is the most i...
212,212,aids,Which medications in the drug class Growth hor...,Effective antiretroviral therapy is the most i...


##### Answer Scraping

In [ ]:
import csv  

header = ['question', 'answer1', 'answer2', 'answer3', 'answer4']

maxi = 0
f = open('aids_ouput.csv', 'w', encoding='UTF8')
writer = csv.writer(f)
# write the header
writer.writerow(header)

for index, row in df.iterrows():
    num = 0
    q = row['Question']
    temp_list = []
    temp_list.append(row['Question'])
    q = q.replace('/','-')
    print(q)
    filename = q + '.txt'
    q = q.replace(' ', '+')
    q = q.replace('?','')
    file_exists = os.path.exists(filename)
    if file_exists:
        continue
    with open(filename, 'a') as the_file:
        the_file.write(row['Question'])
        the_file.write('_a')
        # the_file.write(row['MedScape'])
        # the_file.write('\n')
        nurl = url + q
        nf = requests.get(nurl, headers = headers)
        soup = BeautifulSoup(nf.content,'lxml')
        answers = soup.find_all('div', {'class' : 'cont-text'})
        l = len(answers)
        i = 0
        for anchor in answers:
            ans = anchor.text
            tans = ans.lower()
            d = row['Disease'].lower()
            if tans.find(d) and len(temp_list) <= 5:
                print(" ".join(anchor.text.split()))
                the_file.write(" ".join(anchor.text.split()))
                temp_list.append(" ".join(anchor.text.split()))
                if i < l-1:
                  the_file.write("_a")
                print('\n')
            elif tans.find('hiv') and len(temp_list) <= 5:
                print(" ".join(anchor.text.split()))
                the_file.write(" ".join(anchor.text.split()))
                temp_list.append(" ".join(anchor.text.split()))
                if i < l-1:
                  the_file.write("_a")
                print('\n')
            i = i+1
            num = num + 1
        if(num > maxi):
            maxi = num
        if(len(temp_list) < 5):
            while(len(temp_list) < 5):
                temp_list.append(" ")
        writer.writerow(temp_list)

What is HIV and how is it transmitted?
Human immunodeficiency virus (HIV) is the virus that causes AIDS. When a person becomes infected with HIV, the virus attacks and weakens the immune system. As the immune system weakens, the person is at risk of getting life-threatening infections and cancers. When that happens, the illness is called AIDS. HIV can be transmitted to the fetus or the newborn during pregnancy, during labor or delivery, or by breastfeeding.


Sexually transmitted diseases (STDs) are infections that you can get from having sex with someone who has the infection. The causes of STDs are bacteria, parasites and viruses. There are more than 20 types of STDs, including Chlamydia Gonorrhea Genital herpes HIV/AIDS HPV Syphilis Trichomoniasis Most STDs affect both men and women, but in many cases the health problems they cause can be more severe for women. If a pregnant woman has an STD, it can cause serious health problems for the baby. If you have an STD caused by bacteria or

##### Summarization

In [ ]:
!pip install bert-extractive-summarizer
from summarizer import Summarizer,TransformerSummarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 20.6 MB/s 
     |████████████████████████████████| 163 kB 57.0 MB/s 
     |████████████████████████████████| 7.6 MB 44.7 MB/s 


In [ ]:
text = "Human immunodeficiency virus (HIV) is the virus that causes AIDS. When a person becomes infected with HIV, the virus attacks and weakens the immune system. As the immune system weakens, the person is at risk of getting life-threatening infections and cancers. When that happens, the illness is called AIDS. HIV can be transmitted to the fetus or the newborn during pregnancy, during labor or delivery, or by breastfeeding."

In [ ]:
# BERT - Extractive Summarization
bert_model = Summarizer()
# bert_summary = ''.join(bert_model(text, min_length=60))
# print(bert_summary)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
bert_summary = ''.join(bert_model(text, min_length=60))

In [ ]:
print(bert_summary)

Human immunodeficiency virus (HIV) is the virus that causes AIDS. As the immune system weakens, the person is at risk of getting life-threatening infections and cancers.


##### Add summary to dataset

In [ ]:
import pandas as pd
import csv
import unicodedata

In [ ]:
file = open('/content/aids_ouput.csv', encoding= 'unicode_escape')
csvreader = csv.reader(file)
ansList = []
for i, row in enumerate(csvreader):
    # i is the row number
    if i != 0:
      ans = ""
      for j in range(1, 5):
        if(row[j] != ""):
          # print(row[j])
          ans += str(j) + ". "
          ans += row[j]
          ans += "\n"
      ansList.append(ans)

i = 1
ans_list = []
ans_list.append("Summarized Answer")
for k in ansList:
  if k != "":
    bert_sum = ''.join(bert_model(k, min_length=60))
    # print(str(i) + ". " + bert_sum + "\n")
    ans_list.append(bert_sum)
  i = i+1


FileNotFoundError: ignored

In [ ]:
with open('/content/aids_ouput.csv',  encoding= 'unicode_escape') as read_f:
    with open('AIDS_Summary.csv', 'w', newline='') as write_f:
        csv_reader = csv.reader(read_f)
        csv_writer = csv.writer(write_f)
        i = 0
        for row in csv_reader:
          # if(i < 7):
            row.append(ans_list[i])
            csv_writer.writerow(row)
            i += 1